In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as pyplot
import seaborn as sns

In [2]:
import kagglehub
path = kagglehub.dataset_download("quora/question-pairs-dataset")

100%|██████████| 20.8M/20.8M [00:00<00:00, 78.1MB/s]

Extracting files...


In [3]:
df = pd.read_csv(f"{path}/questions.csv")

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
new_df = df.sample(30000)

In [6]:
new_df.isnull().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,0
is_duplicate,0


In [7]:
new_df.duplicated().sum()

np.int64(0)

In [8]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
211324,Why China is so rich?,Why is China rich?
133275,How did T2 Trainspotting (2017 movie) get gree...,How did T2: Trainspotting (2017 movie) get gre...
229800,How are sex scenes in movies shot? As an actor...,Who are some famous Indian actors who have nev...
381745,What is the full form of etc.?,Why does Gaelic use a Latin alphabet when it i...
399940,If dark energy is created with expansion can i...,If vacuum gravitational and dark energy are cr...


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=3000)
questions = list(ques_df['question1']) + list(ques_df['question2'])
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [13]:
temp_df1 = pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index=ques_df.index)

temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [14]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
211324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
381745,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
399940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [16]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
211324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
133275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
229800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
381745,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
399940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
from sklearn.model_selection import train_test_split

x = temp_df.iloc[:,0:-1].values
y = temp_df.iloc[:,-1].values

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(x_train, y_train)

y_pred = rf.predict(x_test)
accuracy_score(y_test, y_pred)

0.7546666666666667

In [19]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb = XGBClassifier()
xgb.fit(x_train, y_train)

y_pred = xgb.predict(x_test)
accuracy_score(y_test, y_pred)

0.7385